In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import feather
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Colonnes à renommer pour plus de facilités 
col_names = ['code_insee_region', 'region', 'nature', 'date', 'heure', 'date_heure', 'consommation', 'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioenergies', 'ech_physiques', 'flux_physiques_d_auvergne_rhone_alpes_vers_grand_est', 'flux_physiques_de_bourgogne_franche_comte_vers_grand_est', 'flux_physiques_de_bretagne_vers_grand_est', 'flux_physiques_de_centre_val_de_loire_vers_grand_est', 'flux_physiques_de_grand_est_vers_grand_est', 'flux_physiques_de_hauts_de_france_vers_grand_est', 'flux_physiques_d_ile_de_france_vers_grand_est', 'flux_physiques_de_normandie_vers_grand_est', 'flux_physiques_de_nouvelle_aquitaine_vers_grand_est', 'flux_physiques_d_occitanie_vers_grand_est', 'flux_physiques_de_pays_de_la_loire_vers_grand_est', 'flux_physiques_de_paca_vers_grand_est', 'flux_physiques_de_grand_est_vers_auvergne_rhone_alpes', 'flux_physiques_de_grand_est_vers_bourgogne_franche_comte', 'flux_physiques_de_grand_est_vers_bretagne', 'flux_physiques_de_grand_est_vers_centre_val_de_loire', 'flux_physiques_de_grand_est_vers_grand_est0', 'flux_physiques_de_grand_est_vers_hauts_de_france', 'flux_physiques_de_grand_est_vers_ile_de_france', 'flux_physiques_de_grand_est_vers_normandie', 'flux_physiques_de_grand_est_vers_nouvelle_aquitaine', 'flux_physiques_de_grand_est_vers_occitanie', 'flux_physiques_de_grand_est_vers_pays_de_la_loire', 'flux_physiques_de_grand_est_vers_paca', 'flux_physiques_allemagne_vers_grand_est', 'flux_physiques_belgique_vers_grand_est', 'flux_physiques_espagne_vers_grand_est', 'flux_physiques_italie_vers_grand_est', 'flux_physiques_luxembourg_vers_grand_est', 'flux_physiques_royaume_uni_vers_grand_est', 'flux_physiques_suisse_vers_grand_est', 'flux_physiques_de_grand_est_vers_allemagne', 'flux_physiques_de_grand_est_vers_belgique', 'flux_physiques_de_grand_est_vers_espagne' , 'flux_physiques_de_grand_est_vers_italie', 'flux_physiques_de_grand_est_vers_luxembourg', 'flux_physiques_de_grand_est_vers_royaume_uni', 'flux_physiques_de_grand_est_vers_suisse', 'tco_thermique', 'tch_thermique', 'tco_nucleaire', 'tch_nucleaire', 'tco_eolien', 'tch_eolien', 'tco_solaire', 'tch_solaire', 'tco_hydraulique', 'tch_hydraulique', 'tco_bioenergies', 'tch_bioenergies']
# na_values observées dans le jeu de données
df = pd.read_csv('data/energy.csv', sep=';', na_values=['-', 'ND'], header=0, names=col_names)

In [2]:
cols_na = ['consommation', 'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioenergies']
#na_energy = df[df[cols_na].isna().all(1)]

In [3]:
# Supression des colonnes vides
cols_na = df.columns[(df.isna().sum() == df.shape[0])]
df.drop(columns=cols_na, inplace=True)
df.drop(columns=['date', 'heure', 'tco_thermique', 'tch_thermique', 'tco_nucleaire', 'tch_nucleaire', 'tco_eolien', 'tch_eolien', 'tco_solaire', 'tch_solaire', 'tco_hydraulique', 'tch_hydraulique', 'tco_bioenergies', 'tch_bioenergies'], inplace=True)

In [4]:
# Conversion en datetime de date_heure
df.date_heure = pd.to_datetime(df.date_heure, utc=True)


In [5]:
# Récupère que les données sup à 2013
df = df[(df.date_heure.dt.year > 2012) & (df.date_heure.dt.year < 2021)].reset_index(drop=True)

In [6]:
# Ajoute quelques colonnes
df['annee'] = df['date_heure'].dt.year
df['mois'] = df['date_heure'].dt.month
df['jour'] = df['date_heure'].dt.day
df['heure'] = df['date_heure'].dt.hour
df['minute'] = df['date_heure'].dt.minute
df['seconde'] = df['date_heure'].dt.second
df['dayofweek'] = df['date_heure'].dt.dayofweek
df['am'] = df['date_heure'].dt.strftime('%Y-%m')
df['amj'] = df['date_heure'].dt.strftime('%Y-%m-%d')
# 1 = hiver, 2 = printemps, 3 = été, 4 = automne 
df['saison'] = df.mois%12 // 3+1

In [7]:
# Calcul de la production de pompage
# Change le nom de pompage
df['pompage_prod'] = df.pompage.fillna(0)*(1-1.25)

In [8]:
df.rename(columns={'pompage': 'pompage_conso'}, inplace=True)

In [9]:
df['total_prod'] = df.thermique.fillna(0) + df.nucleaire.fillna(0) + df.eolien.fillna(0) + df.solaire.fillna(0) + df.hydraulique.fillna(0) + df.pompage_prod.fillna(0)

In [10]:
cols = ['code_insee_region', 'region', 'nature',
    'date_heure','annee','mois', 'jour', 'heure', 'minute', 'seconde', 'dayofweek', 'am', 'amj', 'saison',
    'consommation', 'ech_physiques',
    'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique',
    'bioenergies', 'pompage_prod', 'pompage_conso', 'total_prod']

In [11]:
# Ordonne les colonnes
df = df[cols]

In [12]:
df.to_feather('data/energy_clean.ftr')

In [13]:
df.columns

Index(['code_insee_region', 'region', 'nature', 'date_heure', 'annee', 'mois',
       'jour', 'heure', 'minute', 'seconde', 'dayofweek', 'am', 'amj',
       'saison', 'consommation', 'ech_physiques', 'thermique', 'nucleaire',
       'eolien', 'solaire', 'hydraulique', 'bioenergies', 'pompage_prod',
       'pompage_conso', 'total_prod'],
      dtype='object')

In [20]:
print(df[ df['region'] == 'Auvergne-Rhône-Alpes']['ech_physiques'].mean())
print(df[ df['region'] == 'Île-de-France']['ech_physiques'].mean())

-5273.854352041981
7665.807673112024
